# AdaptiveMD

## Example 2 - Running of Tasks

### 0. Imports

In [1]:
import sys, os

In [2]:
from adaptivemd import Project, Event, FunctionalEvent, Trajectory

Let's open our `test` project by its name. If you completed the previous example this should all work out of the box.

In [3]:
project = Project('tutorial')

Open all connections to the `MongoDB` and `Session` so we can get started.

Let's see where we are. These numbers will depend on whether you run this notebook for the first time or just continue again. Unless you delete your project it will accumulate models and files over time, as is our ultimate goal.

In [4]:
print project.tasks

print project.trajectories
print project.models

<StoredBundle for with 34 file(s) @ 0x10b13e910>
<ViewBundle for with 7 file(s) @ 0x10b13ea10>
<StoredBundle for with 0 file(s) @ 0x10b13e850>


Now restore our old ways to generate tasks by loading the previously used generators.

In [5]:
engine = project.generators['openmm']
modeller = project.generators['pyemma']
pdb_file = project.files['initial_pdb']

Remember that we stored some files in the database and of course you can look at them again, should that be important.

In [6]:
print pdb_file.get_file()[:1000] + ' [...]'

REMARK   1 CREATED WITH MDTraj 1.8.0, 2016-12-22
CRYST1   26.063   26.063   26.063  90.00  90.00  90.00 P 1           1 
MODEL        0
ATOM      1  H1  ACE A   1      -1.900   1.555  26.235  1.00  0.00          H   
ATOM      2  CH3 ACE A   1      -1.101   2.011  25.651  1.00  0.00          C   
ATOM      3  H2  ACE A   1      -0.850   2.954  26.137  1.00  0.00          H   
ATOM      4  H3  ACE A   1      -1.365   2.132  24.600  1.00  0.00          H   
ATOM      5  C   ACE A   1       0.182   1.186  25.767  1.00  0.00          C   
ATOM      6  O   ACE A   1       1.089   1.407  26.645  1.00  0.00          O   
ATOM      7  N   ALA A   2       0.302   0.256  24.807  1.00  0.00          N   
ATOM      8  H   ALA A   2      -0.588   0.102  24.354  1.00  0.00          H   
ATOM      9  CA  ALA A   2       1.498  -0.651  24.567  1.00  0.00          C   
ATOM     10  HA  ALA A   2       1.810  -0.944  25.570  1.00  0.00          H   
ATOM     11  CB  ALA A   2       1.054  -1.959  23.852

### 1. Run simulations

Before we talk about adaptivity, let's have a look at possibilities to generate trajectories.

We assume that you successfully ran a first trajectory using a worker. Next, we talk about lot's of ways to generate new trajectories.

#### Trajectories from a pdb

You will do this in the beginning. Remember we already have a PDB stored from setting up the engine. if you want to start from this configuration do as before

1. get the trajectory you want
2. make a task
3. submit the task

##### The `Trajectory` object

In [7]:
file_name = next(project.traj_name)

trajectory = Trajectory(
    location=file_name,                          # this creates a new filename
    frame=pdb_file,                              # initial frame is the PDB
    length=100                                   # length is 100 frames
)

Since this is tedious to write there is a shortcut

In [8]:
trajectory = project.new_trajectory(
    frame=pdb_file,
    length=100,
    number=1          # if more then one you get a list of trajectories
)

##### The `Task` object

In [9]:
task_run = engine.task_run_trajectory(trajectory)

This was easy, but we can do some interesting stuff. Since we know the trajectory will exist now we can also extend by some frames.

In [10]:
task_extend = engine.task_extend_trajectory(trajectory, 50)

In [11]:
print task_extend.description

Task: OpenMMEngine [created]

Required : ['sandbox:///{}/00000007/', 'staging:///system.xml', 'staging:///integrator.xml', 'staging:///alanine.pdb', 'staging:///openmmrun.py']
Output : ['sandbox:///{}/00000007/']
Modified : ['sandbox:///{}/00000007/']

<pretask>
Link('staging:///alanine.pdb' > 'worker://initial.pdb)
Link('staging:///system.xml' > 'worker://system.xml)
Link('staging:///integrator.xml' > 'worker://integrator.xml)
Link('staging:///openmmrun.py' > 'worker://openmmrun.py)
Link('sandbox:///{}/00000007/' > 'worker://source/)
Touch('worker://extension/')
python openmmrun.py -r --report-interval 1 -p CPU --store-interval 1 --restart sandbox:///projects/tutorial/trajs/00000007/restart.npz -t worker://initial.pdb --length 50 worker://extension/
mdconvert -o worker://extension/extension.dcd -t worker://initial.pdb worker://source/output.dcd worker://extension/output.dcd
Move('worker://{}/extension.dcd' > 'worker://{}/output.dcd)
Move('worker://extension/' > 'sandbox:///{}/00000007

The only problem is to make sure the tasks are run in the correct order. This would not be a problem if the worker will run tasks in the order they are place in the queue, but that defeats the purpose of parallel runs. Therefore an extended tasks knows that is depends on the existance of the source trajectory. The worker will hence only run a trajectory, once the source exists.

In [12]:
project.queue(task_run, task_extend)

#### Check the results

For a seconds let's see if everything went fine.

In [13]:
for t in project.trajectories:
    print t.short, t.length

sandbox:///{}/00000001/ 100
sandbox:///{}/00000001/ 150
sandbox:///{}/00000003/ 150
sandbox:///{}/00000005/ 100


If this works, then you should see one 100 frame trajectory from the setup (first example) and a second 150 length trajectory that we just generated by running 100 frames and extending it by another 50.

If not, there might be a problem or (more likely) the tasks are not finished yet. Just try the above cell again and see if it changes to the expected output.

Let's do something stupid and produce an error by using a wrong initial pdb file.

In [14]:
trajectory = project.new_trajectory(engine['system_file'], 100)
task = engine.task_run_trajectory(trajectory)
project.queue(task)

Well, nothing changed obviously and we expect it to fail. So let's inspect what happened.

In [15]:
task.state

u'created'

It failed, well, we kind of knew that. No suprise here, but why? Let's look at the stdout and stderr

In [16]:
print task.stdout

None


In [17]:
print task.stderr

None


We see, what we expect. In `openmmrun.py` the openmm executable it could not load the pdb file. 

> *NOTE* If your worker dies for some reason, it will not set a STDOUT or STDERR. If you think that your task should be able to execute, then you can do `task.state = 'created'` and reset it to be accessible to workers. This is NOT recommended, just to explain how this works. Of course you need a new worker anyway.

#### What else

If you have the trajectory object and want to just create the trajectory, you can also put this in the queue. A `Task` object is generated for you, BUT you will have no direct access to the task as in this example. Do whatever you need

In [18]:
project.queue(project.new_trajectory(pdb_file, 100))

### Trajectories from other trajectories

This will be the most common case. At least in any remote kind of adaptivity you will not start always from the same position or extend. You want to pick any exisiting frame and continue from there. So, let's do that.

First we get a trajectory. Every bundle in the project (e.g. `.trajectories`, `.models`, `.files`, `.tasks`) acts like an enhanced set. You can iterate over all entries as we did before, and you can get one element, which usually is the first stored, but not always. For now that is enough

In [19]:
trajectory = project.trajectories.one
print trajectory.path, len(trajectory)

/projects/tutorial/trajs/00000001/ 100


Good, at least 100 frames. We pick, say, frame at index 28 (which is the 29th frame, we start counting at zero) using the way you pick an element from a python list (which is almost what a `Trajectory` represents, a list of frames)

In [20]:
frame = trajectory[28]
print frame

Frame([28])


To run a trajectory just use the frame as the initial frame.

In [21]:
trajectory = project.new_trajectory(frame, 100)
task = engine.task_run_trajectory(trajectory)
project.queue(task)

Btw, you can wait until something happens using `project.wait_until(condition)`. This is not so useful in notebooks, but in scripts it does. `condition` here is a function that evaluates to `True` or `False`. it will be tested in regular intervals and once it is `True` the function returns.

In [22]:
project.wait_until(task.is_done)

Each `Task` has a function `is_done` that you can use. It will return once a task is done. That means it either failed or succeeded or was cancelled. Basically when it is not queued anymore.

If you want to run adaptively, _all you need to do_ is to figure out where to start new simulations from and use the methods provided to run these.

### `Model` tasks

A model generating task work similar to trajectories. You create the generator with options (so far, this will become more complex in the future) and then you create a `Task` from passing it a list of trajectories to be analyzed.

In [13]:
task = modeller.task_run_msm_files(list(project.trajectories.all.file('output.dcd')))
project.queue(task)

In [14]:
print task.description

Task: PyEMMAAnalysis [queued]
Worker: Stevie.fritz.box:/Users/jan-hendrikprinz/Studium/git/adaptivemd
        cd worker.0xe08756070e1711e7aea30000000000f0L

Required : ['staging:///_run_.py', 'file://{}/_rpc_input_0xe08756070e1711e7aea30000000000f0L.json', 'staging:///alanine.pdb']
Output : ['file://{}/_rpc_output_0xe08756070e1711e7aea30000000000f0L.json']
Modified : []

<pretask>
Transfer('file://{}/_rpc_input_0xe08756070e1711e7aea30000000000f0L.json' > 'worker://input.json)
Link('staging:///_run_.py' > 'worker://_run_.py)
Link('staging:///alanine.pdb' > 'worker://input.pdb)
python _run_.py
Transfer('worker://output.json' > 'file://{}/_rpc_output_0xe08756070e1711e7aea30000000000f0L.json)
<posttask>


In [9]:
from adaptivemd import WorkerScheduler

In [10]:
sc = WorkerScheduler(project.resource)
sc.project = project

In [11]:
sc.task_to_script(task)

file:///Users/jan-hendrikprinz/Studium/git/adaptivemd/examples/tutorial/_rpc_input_0xe08756070e1711e7aea30000000000bcL.json
_file__rpc_input_0xe08756070e1711e7aea30000000000bcL.json
worker://input.json
input.json
worker://output.json
output.json
file:///Users/jan-hendrikprinz/Studium/git/adaptivemd/examples/tutorial/_rpc_output_0xe08756070e1711e7aea30000000000bcL.json
_file__rpc_output_0xe08756070e1711e7aea30000000000bcL.json
# Could not write or rename file
# Could not write or rename file
ln -s staging:///_run_.py worker://_run_.py
ln -s ../staging_area/_run_.py _run_.py
ln -s staging:///alanine.pdb worker://input.pdb
ln -s ../staging_area/alanine.pdb input.pdb
python _run_.py
python _run_.py
ln -s output.json _file__rpc_output_0xe08756070e1711e7aea30000000000bcL.json
ln -s output.json _file__rpc_output_0xe08756070e1711e7aea30000000000bcL.json


['set -e',
 '# Could not write or rename file',
 'ln -s ../staging_area/_run_.py _run_.py',
 'ln -s ../staging_area/alanine.pdb input.pdb',
 'python _run_.py',
 'ln -s output.json _file__rpc_output_0xe08756070e1711e7aea30000000000bcL.json']

In [17]:
for m in project.models:
    print m

So we generated one model. The `Model` objects contain (in the base version) only a `.data` attribute which is a dictionary of information about the generated model.

In [18]:
model = project.models.last
model.data.keys()

TypeError: 'NoneType' object has no attribute '__getitem__'

In [32]:
print model.data['msm']['P']

[[ 0.88372093  0.03082308  0.          0.05469591  0.03076008]
 [ 0.06062159  0.86170213  0.02155719  0.02729711  0.02882198]
 [ 0.          0.08580974  0.82608696  0.08810329  0.        ]
 [ 0.08060759  0.02045439  0.01658508  0.88235295  0.        ]
 [ 0.0796843   0.03796276  0.          0.          0.88235293]]


### Pick frames automatically

The last thing that is implemented is a function that can utilize models to decide which frames are better to start from. The simplest one will use the counts per state, take the inverse and use this as a distribution.

In [33]:
project.find_ml_next_frame(4)

[Frame(00000000.dcd[59]),
 Frame(00000000.dcd[35]),
 Frame(00000003.dcd[98]),
 Frame(00000000.dcd[55])]

So you can pick states according to the newest (last) model. (This will be moved to the Brain). And since we want trajectories with these frames as starting points there is also a function for that

In [34]:
trajectories = project.new_ml_trajectory(length=100, number=4)
trajectories

[Trajectory(Frame(00000002.dcd[134]) >> 00000021.dcd[0..100]),
 Trajectory(Frame(00000003.dcd[95]) >> 00000022.dcd[0..100]),
 Trajectory(Frame(00000000.dcd[77]) >> 00000023.dcd[0..100]),
 Trajectory(Frame(00000000.dcd[91]) >> 00000024.dcd[0..100])]

Let's submit these before we finish this notebook with a quick discussion of workers

In [35]:
project.queue(trajectories)

That's it.

### What about workers

Worker are the instances that execute tasks for you. If you did not stop the worker it will still be running and you can check its state

In [36]:
from adaptivemd import DT

DT is a little helper to convert time stamps into something readable.

In [44]:
project.trigger()
for w in project.workers:
    if w.state == 'running':
        print '[%s:%s] %s:%s runs `%s` tasks' % (w.state, DT(w.seen).time, w.hostname, w.cwd, w.n_tasks)

[running:11:20:36] Stevie.fritz.box:/Users/jan-hendrikprinz/Studium/git/adaptivemd runs `0` tasks


Okay, the worker is running, was last reporting its heartbeat at ... and has a hostname and current working directory (where it was executed from). The generators specify which tasks from some generators are executed. If it is `None` then the worker runs all tasks it finds. You can use this to run specific workers for models and some for trajectory generation.

You can also controle it remotely by sending it a command. `shutdown` will shut it down for you.

In [50]:
# project.workers.last.command = 'shutdown'

Afterwards you need to restart you worker to continue.

In [46]:
project.close()